Data upload and preprocessing

In [2]:
import pandas as pd
import numpy as np


In [3]:
from google.colab import files

In [5]:
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data.csv


In [23]:
df = pd.read_csv('1pharma-data.csv')

In [24]:
df.head()

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product_ID,Product Class,Quantity,Price,Sales,Month,Year,Name of Sales Rep,Manager,Sales Team
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta


Data Exploration and Cleaning

In [25]:
print(df.columns.tolist())


['Distributor', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product_ID', 'Product Class', 'Quantity', 'Price', 'Sales', 'Month', 'Year', 'Name of Sales Rep', 'Manager', 'Sales Team']


In [26]:
df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude'], axis=1, inplace=True)

In [27]:
print(df['Month'].unique())

['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']


In [28]:
#print(df['Product_ID'].unique())

Feature Engineering

Dataset Scaling

In [30]:
numeric_cols = df.select_dtypes(include=np.number).columns

In [31]:
from sklearn.preprocessing import MinMaxScaler

In [32]:
scaler = MinMaxScaler()

In [33]:
df_scaled = scaler.fit_transform((df[numeric_cols]))

In [34]:
df1 = pd.DataFrame(df_scaled, columns=numeric_cols)

In [35]:
df1 = df1.join(df.select_dtypes(exclude=np.number))

In [36]:
df1.head()

,Quantity,Price,Sales,Year,Distributor,Product_ID,Month
0,0.057724,0.448187,0.053123,0.333333,Gottlieb-Cruickshank,Topipizole,January
1,0.057748,0.737047,0.053157,0.333333,Gottlieb-Cruickshank,Choriotrisin,January
2,0.057933,0.056995,0.053129,0.333333,Gottlieb-Cruickshank,Acantaine,January
3,0.057740,0.534974,0.053137,0.333333,Gottlieb-Cruickshank,Lioletine Refliruvax,January
4,0.057853,0.564767,0.053221,0.333333,Gottlieb-Cruickshank,Oxymotroban Fexoformin,January


Split Dataset

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
df1.dropna(axis=0, subset=['Quantity'], inplace=True)
y = df1.Quantity
X = df1.drop(['Quantity'], axis=1).select_dtypes(exclude=['object'])

In [39]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
X

,Price,Sales,Year
0,0.448187,0.053123,0.333333
1,0.737047,0.053157,0.333333
2,0.056995,0.053129,0.333333
3,0.534974,0.053137,0.333333
4,0.564767,0.053221,0.333333
...,...,...,...
254077,0.615285,0.058932,1.000000
254078,0.853627,0.056858,1.000000
254079,0.849741,0.055872,1.000000
254080,0.120466,0.053933,1.000000


In [41]:
y

,Quantity
0,0.057724
1,0.057748
2,0.057933
3,0.057740
4,0.057853
...,...
254077,0.065056
254078,0.061154
254079,0.060256
254080,0.062220


XGBoost Algorithm

In [42]:
from xgboost import XGBRegressor

In [43]:
model = XGBRegressor()

In [44]:
model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [45]:
model.predict(test_X)

array([0.05774918, 0.05776008, 0.05770369, ..., 0.05790919, 0.05786351,
       0.05770369], dtype=float32)

In [46]:
y_predict = model.predict(test_X)

In [47]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error

In [48]:
mean_squared_error(test_y, y_predict)

1.8884921041419135e-05

In [49]:
mean_absolute_error(test_y, y_predict)

0.00020023911119813092

In [50]:
 mean_absolute_percentage_error(test_y, y_predict)

0.0020542368839592696

In [51]:
 root_mean_squared_error(test_y, y_predict)

0.004345678432813355

In [52]:
pip install m5-wrmsse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.8 MB/s eta 0:00:00


In [57]:
from m5_wrmsse import wrmsse

In [61]:
wrmsse = np.(test_y, _predict)((30490,28))

SyntaxError: invalid syntax (<ipython-input-61-1f99a1c602cc>, line 1)

In [64]:
wrmsse_score = wrmsse(test_y, y_pred)

TypeError: wrmsse() takes 1 positional argument but 2 were given

In [63]:
print(wrmsse_score)

2.563051076566426


Model tunning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [ ]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [ ]:
grid_search.fit(train_X, train_y)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

In [ ]:
grid_search.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0,
 'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 800,
 'subsample': 1.0}

In [ ]:
grid_search.best_estimator_.predict(test_X)

array([0.0577478 , 0.0577748 , 0.05770995, ..., 0.05788808, 0.05779059,
       0.05768708], dtype=float32)

In [ ]:
y_pred_tuned = grid_search.best_estimator_.predict(test_X)

In [ ]:
mean_squared_error(test_y, y_pred_tuned)

1.2432490322433438e-05

In [ ]:
mean_absolute_error(test_y, y_pred_tuned)

0.0002200053172871931

In [ ]:
mean_absolute_percentage_error(test_y, y_pred_tuned)

0.002448982142046014

In [ ]:
 root_mean_squared_error(test_y, y_pred_tuned)

0.003525973670127648

In [ ]:
from m5_wrmsse import wrmsse

In [ ]:
y_pred_tuned = np.ones((30490,28))

In [ ]:
wrmsse_score = wrmsse(y_pred_tuned)
print(wrmsse_score)

2.563051076566426


In [ ]:
month_to_int = {
    'January': 1,'February': 2,'March': 3,
    'April': 4,'May': 5,'June': 6,'July': 7,
    'August': 8,'September': 9,'October': 10,
    'November': 11,'December': 12
}

df['Month'] = df['Month'].map(month_to_int)

df.head()


Using the model to forecast